# LIV Curves Analysis
This notebook demonstrates how to load, process, and analyze IV (Current–Voltage) and LIV (Light–Current–Voltage) curves for photonic devices.
It guides you through instrument setup, configuration loading, measurement execution, data visualization, and saving results for further analysis.

---

## Step 1 – Import Libraries
Begin by importing the Python libraries required for instrument control, data analysis, and plotting.

In [ ]:
import pyvisa as visa
from instruments.agilent8163 import Agilent8163Multimeter
from instruments.keithley2400 import Keithley2400SourceMeter
from controllers.laser_sweep import *
from controllers.sourcemeter_sweep import measure_iv_curve, measure_laser_liv_curve_by_source_current
from utils.load_config import load_config
from utils.plotter import plot_measurements
from utils.data_saver import save_raw_measurements

In [ ]:
from utils.visa_utils import list_visa_resources

# List all VISA resources
list_visa_resources()

## Step 2 – Load Instrument Configuration
Load the instrument configuration from a YAML file. This ensures the sweep runs with correct parameters and safety limits for each device.

In [ ]:
# They are connected on the same GPIB bus, so we need to pass the same resource manager to both instruments
# If you are using multiple GPIB buses, then you need to create separate resource managers for each bus
rm = visa.ResourceManager()

# Generate the laser and keithley objects using the configuration
config = load_config('configs/experiment_config.yaml')
keithley_cfg = config["Keithley2400"]                               # Extract Keithley instrument configuration
laser_cfg = config["Agilent8163Multimeter"]                         # Extract laser-specific configuration


keithley_obj = Keithley2400SourceMeter(address=keithley_cfg['address'], resource_manager=rm)
# Generate the laser and keithley objects using the configuration
laser_obj = Agilent8163Multimeter(
    address=laser_cfg['address'],
    laser_slot=laser_cfg['laser_slot'],
    power_slot= laser_cfg['power_slot'],
    power_channel=laser_cfg['power_channel'],
    resource_manager=rm
)




combined_name = f"{config['DUT']['type']}_{config['DUT']['name']}"  # Combined name for saving data

## Step 3 – Define Sweep Parameters and Execute Measurement
Set up the sweep parameters for the LIV measurement and run the experiment to collect data from the instruments.

In [ ]:
liv_params = {
    "sourcemeter": keithley_obj,
    "powermeter": laser_obj,
    "start_current": float(keithley_cfg["start_current"]),
    "stop_current": float(keithley_cfg["stop_current"]),
    "step_current": float(keithley_cfg["step_current"]),
    "measure_voltage_range": keithley_cfg["measure_voltage_range"],
    "voltage_limit": keithley_cfg["compliance_voltage"],
    "wire_mode": keithley_cfg["wire_mode"],
    "center_wavelength": 1310e-9,
    "sourcemeter_delay": keithley_cfg["delay"],
    "powermeter_delay": laser_cfg["delay"],
    "logger": None,
}

headers_liv_sweep, results_liv_sweep = measure_laser_liv_curve_by_source_current(**liv_params)

## Step 4 – Plot Measured IV and LIV Curves
Visualize the measured IV and LIV data to inspect device performance and identify key characteristics.

In [ ]:
# Plots the LIV Resukts
# Trim headers (drop voltage)
trimmed_headers = (headers_liv_sweep[1],headers_liv_sweep[2])  # -> ("Voltage (V)", "Optical Power (dBm)")
# Trim data
trimmed_results = [(c, l) for _, c, l in results_liv_sweep]  # -> List[Tuple[float, float]]

plot_path = plot_measurements(
    headers=trimmed_headers,
    results=[res[:2] for res in trimmed_results],
    figure_name=combined_name + "_Sourcemeter_Sweep_LIV_results", 
    show=True
)

In [ ]:
# Save the laser sweep results to a CSV file
data_path = save_raw_measurements(headers=headers_liv_sweep,
                      data=results_liv_sweep,
                      filename=combined_name + "_Sourcemeter_Sweep_LIV_results",
)

---
# Conclusion

This notebook walked through the complete workflow for IV and LIV curve measurement:
- Loaded instrument configuration to set up a safe and reproducible measurement environment.
- Defined sweep parameters and executed the measurement to capture IV and LIV data.
- Plotted the measured curves for visual inspection and analysis.
- Saved the data for further analysis, such as extracting device parameters or comparing different samples.